### Installs

In [ ]:
!sudo pip install kaggle

### Download the data

In [6]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/home/jupyter/kaggle-competitions/tfqa/config/.kaggle'
!chmod 600 $KAGGLE_CONFIG_DIR/kaggle.json
os.chdir('/home/jupyter/kaggle-competitions/tfqa/data')
!kaggle competitions download tensorflow2-question-answering -f sample_submission.csv

  0%|                                               | 0.00/18.2k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 18.2k/18.2k [00:00<00:00, 13.0MB/s]


In [7]:
!kaggle competitions download tensorflow2-question-answering -f simplified-nq-train.jsonl
!kaggle competitions download tensorflow2-question-answering -f simplified-nq-test.jsonl

100%|█████████████████████████████████████▉| 4.46G/4.46G [00:21<00:00, 70.7MB/s]
100%|███████████████████████████████████████| 4.46G/4.46G [00:21<00:00, 222MB/s]
  0%|                                               | 0.00/4.78M [00:00<?, ?B/s]
100%|███████████████████████████████████████| 4.78M/4.78M [00:00<00:00, 240MB/s]


In [11]:
!unzip simplified-nq-train.jsonl
!unzip simplified-nq-test.jsonl

Archive:  simplified-nq-train.jsonl.zip
  inflating: simplified-nq-train.jsonl  
Archive:  simplified-nq-test.jsonl.zip
  inflating: simplified-nq-test.jsonl  


### Imports

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
path = '/home/jupyter/kaggle-competitions/tfqa/data/'
train_path = 'simplified-nq-train.jsonl'
test_path = 'simplified-nq-test.jsonl'
sample_submission_path = 'sample_submission.csv'

def read_data(path, sample = True, chunksize = 30000):
    if sample == True:
        df = []
        with open(path, 'rt') as reader:
            for i in range(chunksize):
                df.append(json.loads(reader.readline()))
        df = pd.DataFrame(df)
        print('Our sampled dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
    else:
        df = pd.read_json(path, orient = 'records', lines = True)
        print('Our dataset have {} rows and {} columns'.format(df.shape[0], df.shape[1]))
        gc.collect()
    return df

train = read_data(path+train_path, sample = True)
test = read_data(path+test_path, sample = False)
train.head()